In [1]:
import os
import re
import csv
import sys
import ast 
import json
import time
import urllib
import random
import spotipy
import requests
import datetime
import configparser
import numpy as np
import pandas as pd
import urllib.parse
from bs4 import BeautifulSoup
from user_agent import generate_user_agent
from spotipy.oauth2 import SpotifyClientCredentials

In [75]:
list_next_page = []
url1 = 'https://www.thefamouspeople.com/pop-singers.php'
page1 = requests.get(url1)
soup1 = BeautifulSoup(page1.text, "html.parser")
btn = soup1.find('div', class_='pagination')
btn_next_li = btn.find_all('li', class_='nextpage')
page_href = btn_next_li[0].a['href']
for i in range(2,7):
    next_page_url = 'https://www.thefamouspeople.com' + page_href
    next_page_url2 = next_page_url.replace(next_page_url[-1],str(i))
    list_next_page.append(next_page_url2)
list_next_page.insert(0,url1)
list_next_page

['https://www.thefamouspeople.com/pop-singers.php',
 'https://www.thefamouspeople.com/pop-singers.php?page=2',
 'https://www.thefamouspeople.com/pop-singers.php?page=3',
 'https://www.thefamouspeople.com/pop-singers.php?page=4',
 'https://www.thefamouspeople.com/pop-singers.php?page=5',
 'https://www.thefamouspeople.com/pop-singers.php?page=6']

In [87]:
# 抓西洋流行音樂的歌手清單
singer_list = []
subtitle_list = []
for num in range(len(list_next_page)):
    url2 = list_next_page[num]
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36'}
    page2 = requests.get(url2, headers=headers)
    soup2 = BeautifulSoup(page2.text, "html.parser")
    select = soup2.findAll('span',{'class':'btn-primary-cat-new btn-sec-cat'})
    print(url2,':',len(select))
    for i in range(len(select)):
        try:
            singer = select[i].a.text
            subtitle = select[i].div.text
            singer = singer.replace("’","'")
            singer_list.append(singer)
            subtitle_list.append(subtitle)
        except:
            pass
df_singer = pd.DataFrame(list(zip(singer_list,subtitle_list)),columns=['Artist','Subtitle'])
df_singer.sort_values(['Artist','Subtitle'],inplace=True)
df_singer.reset_index(drop=True,inplace=True)
df_singer.to_excel('csv/20200303/artist_list2.xlsx',index=0)
df_singer

https://www.thefamouspeople.com/pop-singers.php : 94
https://www.thefamouspeople.com/pop-singers.php?page=2 : 94
https://www.thefamouspeople.com/pop-singers.php?page=3 : 94
https://www.thefamouspeople.com/pop-singers.php?page=4 : 94
https://www.thefamouspeople.com/pop-singers.php?page=5 : 94
https://www.thefamouspeople.com/pop-singers.php?page=6 : 89


,Artist,Subtitle
0,AJ Michalka,American
1,AJ Mitchell,American
2,Aaron Carter,American
3,Aaron Lohr,American
4,Abraham Mateo,Spanish
...,...,...
554,Zabdiel De Jesus,Puerto Rican
555,Zach Herron,American
556,Zara Larsson,Swedish
557,Zendaya Maree Stoermer Coleman,American


In [88]:
# 去除Subtitle為Japanese、Thai、Nigerien、Hong Konger、South Korean、Chinese
string = 'Japanese|Thai|Nigerien|Hong Konger|South Korean|Chinese'
df_singer_select = df_singer.loc[~df_singer['Subtitle'].str.contains(string)].reset_index(drop=True)
df_singer_select.to_excel('csv/20200303/artist_list2_select.xlsx',index=0)
df_singer_select

,Artist,Subtitle
0,AJ Michalka,American
1,AJ Mitchell,American
2,Aaron Carter,American
3,Aaron Lohr,American
4,Abraham Mateo,Spanish
...,...,...
529,Zabdiel De Jesus,Puerto Rican
530,Zach Herron,American
531,Zara Larsson,Swedish
532,Zendaya Maree Stoermer Coleman,American
